In [ ]:
# Mount your Google Drive if needed
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Note: Some libraries or dependencies used in this code may become deprecated over time.
# If you encounter deprecated libraries, replace them with suitable alternatives and
# update the corresponding sections of the code that rely on the deprecated classes or methods accordingly.

!pip install transformers[torch]==4.31.0
!pip install datasets

import os
import json
import gzip
import pandas as pd
import collections
import nltk
from google.colab import files

from urllib.request import urlopen

import random
import numpy as np
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
import pickle
from IPython.display import clear_output
import torch
from transformers import RobertaTokenizerFast
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support,  recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef
from sklearn.model_selection import train_test_split,KFold, StratifiedKFold
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_metric



In [ ]:

if torch.cuda.is_available():
    print(torch.cuda.device_count())
    print(torch.cuda.get_device_name(0))

# Training

In [ ]:
def set_seed(SEED):
    """ Set random seed to all """
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True



class AmazonDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels= None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
           item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])




def prepare_auxilary_datasets(dataset):

    X_test =  dataset["sentence"]
    y_test =  dataset["label"]   if "label" in dataset.columns.values else   dataset["helpful"]
    # Encoding
    test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=40)
    test_dataset = AmazonDataset(test_encodings, y_test.tolist())
    return test_dataset





def compute_metrics(eval_pred):

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=predictions)
    recall = recall_score(y_true=labels, y_pred=predictions)
    precision = precision_score(y_true=labels, y_pred=predictions)
    f1 = f1_score(y_true=labels, y_pred=predictions)
    macro_f1 = f1_score(y_true=labels, y_pred=predictions, average= 'macro')
    mcc = matthews_corrcoef(labels, predictions)
    auc = roc_auc_score(y_true= labels, y_score= logits[:, 1])
    tn, fp, fn, tp = confusion_matrix(y_true= labels, y_pred=predictions).ravel()
    specificity = tn / (tn+fp)

    return {
        'accuracy'    : accuracy,
        'f1'          : f1,
        'macro_f1'    : macro_f1,
        'precision'   : precision,
        'recall'      : recall,
        'specificity' : specificity,
        'mcc'         : mcc,
        'AUC'         : auc
    }



def return_training_args():

    training_args = TrainingArguments(
        output_dir="test-amazon",
        evaluation_strategy = "epoch",  # there would be no evaluation during training when doing simple K-fold cross validation a
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=256,
        num_train_epochs = 6,
        weight_decay=0.01,
        load_best_model_at_end=False,
        logging_strategy = "steps",
        logging_steps = 50,
        save_strategy= "no",
    )
    return training_args




In [ ]:
frequency = pd.read_pickle('/content/drive/MyDrive/paper1/datasets/dictionary frequency of occurence of needs')

# In domain dataset
data_set_1 = pd.read_csv('/content/drive/MyDrive/Fairness paper datasets/8000_sentences_with_need_IDs.csv')

# Out of domain datasets
data_set_2 = pd.read_csv('/content/drive/MyDrive/Fairness paper datasets/Electronics.csv')
data_set_3 = pd.read_csv('/content/drive/MyDrive/Fairness paper datasets/baby.csv')
data_set_4 = pd.read_csv('/content/drive/MyDrive/Fairness paper datasets/Pet_supplies.csv')
data_set_5 = pd.read_csv('/content/drive/MyDrive/Fairness paper datasets/Sport_outdoors.csv')

# Ensure the 'sentence' column is of type 'str'
data_set_1['sentence'] = data_set_1['sentence'].astype(str)
data_set_2['sentence'] = data_set_2['sentence'].astype(str)
data_set_3['sentence'] = data_set_3['sentence'].astype(str)
data_set_4['sentence'] = data_set_4['sentence'].astype(str)
data_set_5['sentence'] = data_set_5['sentence'].astype(str)

ratios = {
           'A': 0.50,
           'B': 0.20,
           'C': 0.10,
           'D': 0.05,
           'E': 0.025,
           'F': 0.01
           }

models = {
              'albert_base' :"albert-base-v2",
              'distilbert'  :"distilbert-base-uncased",
              'bertB'       :"bert-base-uncased",
              'bertL'       :"bert-large-uncased",
              'robertB'     :"roberta-base",
              'robertL'     :"roberta-large",
              'allenaiB'    :"allenai/reviews_roberta_base",

              'ALBERT-L'      :"albert/albert-large-v2",
              'DISRoBERTa-B'  :"distilbert/distilroberta-base",
              'XLNet-B'       :"xlnet/xlnet-base-cased",
              'XLNet-L'       :"xlnet/xlnet-large-cased",
              'DEBERT-B'      :"microsoft/deberta-base",
              'DEBERT-L'      :"microsoft/deberta-large",
              'XLM-B'         :"FacebookAI/xlm-roberta-base",
              'XLM-L'         :"FacebookAI/xlm-roberta-large"
          }

seeds = [94, 791, 5, 6932, 1759, 323, 1694, 9741, 200, 999]  # randomly selected
all_datasets_ratio_models_seed_results = {}
results = []
results_exclude = []
results_garbage = []
results_uninf_exc = []



for ratio_label, ratio in ratios.items():
    all_datasets_ratio_models_seed_results.update({ratio:{}})

    for model_name, model_address in models.items():
        tokenizer = AutoTokenizer.from_pretrained(model_address)
        all_datasets_ratio_models_seed_results[ratio].update({model_name:{}})
        training_args = return_training_args()  # to reinitialize training arguments:  must be added
        if (ratio < 0.20) and (ratio >= 0.05):
           training_args.per_device_train_batch_size = 8
           training_args.logging_steps = 25
        elif (ratio < 0.05):
           training_args.per_device_train_batch_size = 4
           training_args.logging_steps = 10


        if model_name == 'bertL':
           training_args.learning_rate = 2e-5
        if model_name == 'robertL':
           training_args.learning_rate = 1.25e-5
        if model_name == 'ALBERT-L' :
           training_args.learning_rate = 8e-6
        if model_name == 'DISRoBERTa-B':
           training_args.learning_rate = 2e-5
        if model_name == 'XLNet-B':
           training_args.learning_rate = 2e-5
        if model_name == 'XLNet-L' :
           training_args.learning_rate = 1e-5
        if model_name == 'DEBERT-B':
           training_args.learning_rate = 3e-5
        if model_name == 'DEBERT-L':
           training_args.learning_rate = 1e-5
        if model_name ==  'XLM-B':
           training_args.learning_rate = 2e-5
        if model_name ==  'XLM-L':
           training_args.learning_rate = 8e-6
        else:
           training_args.learning_rate = 2e-5

        for seed in seeds:
            set_seed(seed)
            training_args.seed = seed

            r = int(ratio * data_set_1.shape[0] / 2)
            df_train_uninf = data_set_1[data_set_1.label == 0].sample(n = r, random_state = seed)
            df_train_inf = data_set_1[data_set_1.label == 1].sample(n = r, random_state = seed)
            df_train = pd.concat([df_train_inf, df_train_uninf]).sample(frac= 1, random_state = seed)
            # Find the rows not selected
            df_test_all = data_set_1.drop(df_train.index)
            df_val_uninf = df_test_all[df_test_all.label == 0].sample(n = 400, random_state = seed)
            df_val_inf = df_test_all[df_test_all.label == 1].sample(n = 400, random_state = seed)
            df_val  = pd.concat([df_val_inf, df_val_uninf]).sample(frac= 1, random_state = seed)
            df_test = df_test_all.drop(df_val.index)

            # Reset index
            df_train = df_train.reset_index(drop=True)
            df_val   = df_val.reset_index(drop=True)
            df_test  = df_test.reset_index(drop=True)
            # Encoding
            train_main_encodings = tokenizer(df_train.sentence.tolist(), truncation=True, padding=True, max_length=40)
            val_main_encodings   = tokenizer(df_val.sentence.tolist(), truncation=True, padding=True, max_length=40)
            test_main_encodings  = tokenizer(df_test.sentence.tolist(), truncation=True, padding=True, max_length=40)


            train_dataset     = AmazonDataset(train_main_encodings, df_train.label.tolist())
            val_dataset       = AmazonDataset(val_main_encodings, df_val.label.tolist())
            test_main_dataset = AmazonDataset(test_main_encodings, df_test.label.tolist())


            print(f'Model seed is: {training_args.seed}, total number of '
                  f'training (for each informative and uninform class) and test samples: '
                  f'{df_train.shape[0]}, {df_test.shape[0]}', ratio, model_name,
                  training_args.seed, training_args.learning_rate,
                  training_args.per_device_train_batch_size)
            model = AutoModelForSequenceClassification.from_pretrained(model_address) # option 1: allenai/reviews_roberta_base  # option2: roberta-base
            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset = train_dataset,
                eval_dataset = val_dataset,             # evaluation dataset must be off when doing simple K-fold cross validation
                # callbacks= [EarlyStoppingCallback(early_stopping_patience= 3,early_stopping_threshold=0.05)],
                compute_metrics = compute_metrics
            )

            trainoutput = trainer.train()
            tr_loss = trainoutput.training_loss
            pred = trainer.predict(test_main_dataset)
            results.append(["oral-care", ratio, model_name, seed, pred.metrics['test_accuracy'], pred.metrics['test_f1'], pred.metrics['test_macro_f1'],
                            pred.metrics['test_precision'], pred.metrics['test_recall'], pred.metrics['test_specificity'],
                            pred.metrics['test_mcc'], pred.metrics['test_AUC']])
            log_history = trainer.state.log_history
            all_datasets_ratio_models_seed_results[ratio][model_name].update(
                                                                           { seed : {
                                                                                            "oral-care":            [df_train.Sentence_ID.values, df_val.Sentence_ID.values, df_test.Sentence_ID.values, pred, tr_loss, log_history],
                                                                                            "electronics":          trainer.predict(prepare_auxilary_datasets(data_set_2)),
                                                                                            "baby":                 trainer.predict(prepare_auxilary_datasets(data_set_3)),
                                                                                            "pet-supplies":         trainer.predict(prepare_auxilary_datasets(data_set_4)),
                                                                                            "Sport-outdoors":       trainer.predict(prepare_auxilary_datasets(data_set_5)),
                                                                                          }
                                                                           })
        # Specify the path in Google Drive
        file_path = f'/content/drive/MyDrive/paper1/Sample_Efficiency/all_results_{ratio_label}_{model_name}'
        # Serialize and save the dictionary
        with open(file_path, 'wb') as handle:
             pickle.dump(all_datasets_ratio_models_seed_results, handle, protocol=pickle.HIGHEST_PROTOCOL)




df_result = pd.DataFrame(results, columns = ['category', 'ratio' ,'model', 'seed', 'accuracy','f1-score', 'macro_f1', 'precision', 'recall', 'specificity', 'mcc', 'AUC'])
display(df_result)
display(df_result.groupby(df_result.category).mean(numeric_only = True))


tables = {"oral-care": results,"electronics": [],"baby": [],
          "pet-supplies": [],"Sport-outdoors": []}

for rto, values in all_datasets_ratio_models_seed_results.items():
    for model_name, value in values.items():
        for seed, val in value.items():
            for dataset, v in val.items():
                if dataset == "oral-care":
                  tables["oral-care"] =  df_result
                else:
                   tables[dataset].append([dataset, rto, model_name, seed, v.metrics['test_accuracy'],
                                           v.metrics['test_f1'], v.metrics['test_macro_f1'], v.metrics['test_precision'],
                                           v.metrics['test_recall'], v.metrics['test_specificity'],
                                           pred.metrics['test_mcc'], v.metrics['test_AUC']]
                                          )


df_total_list = [pd.DataFrame(t, columns = ['category','ratio' ,'model', 'seed', 'accuracy','f1-score', 'macro_f1','precision', 'recall', 'specificity', 'mcc', 'AUC']) for k, t in tables.items()]
df = pd.concat(df_total_list).drop_duplicates().reset_index(drop=True)
display(df.groupby(df.category).mean(numeric_only = True))




/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

...

Model seed is: 999, total number of training (for each informative and uninform class) and test samples: 80, 7120 0.01 DEBERT-B 999 2e-05 4


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Macro F1,Precision,Recall,Specificity,Mcc,Auc
1,0.721600,0.696636,0.500000,0.666667,0.333333,0.500000,1.000000,0.000000,0.000000,0.581244
2,0.688800,0.694256,0.500000,0.666667,0.333333,0.500000,1.000000,0.000000,0.000000,0.623931
3,0.690000,0.689817,0.607500,0.668076,0.593977,0.578755,0.790000,0.425000,0.230933,0.646100
4,0.584900,0.671983,0.590000,0.677165,0.557761,0.558442,0.860000,0.320000,0.213862,0.674447
5,0.536500,0.898099,0.623750,0.531882,0.608679,0.703704,0.427500,0.820000,0.269094,0.695794
6,0.288300,0.811777,0.638750,0.629012,0.638501,0.646438,0.612500,0.665000,0.277883,0.698738


,category,ratio,model,seed,accuracy,f1-score,macro_f1,precision,recall,specificity,mcc,AUC
0,oral-care,0.10,ALBERT-L,94,0.528438,0.691222,0.346933,0.528390,0.999113,0.001325,0.006609,0.621835
1,oral-care,0.10,ALBERT-L,791,0.678906,0.656182,0.677497,0.755393,0.580006,0.789665,0.375835,0.753581
2,oral-care,0.10,ALBERT-L,5,0.738906,0.761864,0.736457,0.735149,0.790594,0.681020,0.475292,0.783641
3,oral-care,0.10,ALBERT-L,6932,0.749219,0.759623,0.748748,0.769417,0.750074,0.748261,0.497761,0.814214
4,oral-care,0.10,ALBERT-L,1759,0.637656,0.688265,0.627848,0.630853,0.757172,0.503809,0.270467,0.667575
...,...,...,...,...,...,...,...,...,...,...,...,...
195,oral-care,0.01,DEBERT-B,323,0.649719,0.655048,0.649635,0.678704,0.632986,0.668245,0.300901,0.713786
196,oral-care,0.01,DEBERT-B,1694,0.652247,0.671269,0.651079,0.666843,0.675755,0.626221,0.302222,0.702237
197,oral-care,0.01,DEBERT-B,9741,0.634972,0.670220,0.630754,0.637923,0.705961,0.556378,0.265547,0.685477
198,oral-care,0.01,DEBERT-B,200,0.653090,0.681783,0.650246,0.658045,0.707298,0.593075,0.302531,0.701000


TypeError: Could not convert ALBERT-LALBERT-LALBERT-LALBERT-LALBERT-LALBERT-LALBERT-LALBERT-LALBERT-LALBERT-LDISRoBERTa-BDISRoBERTa-BDISRoBERTa-BDISRoBERTa-BDISRoBERTa-BDISRoBERTa-BDISRoBERTa-BDISRoBERTa-BDISRoBERTa-BDISRoBERTa-BXLNet-BXLNet-BXLNet-BXLNet-BXLNet-BXLNet-BXLNet-BXLNet-BXLNet-BXLNet-BXLNet-LXLNet-LXLNet-LXLNet-LXLNet-LXLNet-LXLNet-LXLNet-LXLNet-LXLNet-LDEBERT-BDEBERT-BDEBERT-BDEBERT-BDEBERT-BDEBERT-BDEBERT-BDEBERT-BDEBERT-BDEBERT-BALBERT-LALBERT-LALBERT-LALBERT-LALBERT-LALBERT-LALBERT-LALBERT-LALBERT-LALBERT-LDISRoBERTa-BDISRoBERTa-BDISRoBERTa-BDISRoBERTa-BDISRoBERTa-BDISRoBERTa-BDISRoBERTa-BDISRoBERTa-BDISRoBERTa-BDISRoBERTa-BXLNet-BXLNet-BXLNet-BXLNet-BXLNet-BXLNet-BXLNet-BXLNet-BXLNet-BXLNet-BXLNet-LXLNet-LXLNet-LXLNet-LXLNet-LXLNet-LXLNet-LXLNet-LXLNet-LXLNet-LDEBERT-BDEBERT-BDEBERT-BDEBERT-BDEBERT-BDEBERT-BDEBERT-BDEBERT-BDEBERT-BDEBERT-BALBERT-LALBERT-LALBERT-LALBERT-LALBERT-LALBERT-LALBERT-LALBERT-LALBERT-LALBERT-LDISRoBERTa-BDISRoBERTa-BDISRoBERTa-BDISRoBERTa-BDIS

In [ ]:

# Specify the path in Google Drive
file_path = f'/content/drive/MyDrive/paper1/temp/Sample_Efficiency/all_results_sample_efficiency_after_revision_3'
# Serialize and save the dictionary
with open(file_path, 'wb') as handle:
     pickle.dump(all_datasets_ratio_models_seed_results, handle, protocol=pickle.HIGHEST_PROTOCOL)